In [1]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [2]:
tel = pd.read_csv('data/telecom_data.csv', sep=';')
tel.head()

,index,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,5575-GNVDE,Male,No,No,No,34,Yes,No,DSL,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
1,2,3668-QPYBK,Male,No,No,No,2,Yes,No,DSL,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
2,3,7795-CFOCW,Male,No,No,No,45,No,No phone service,DSL,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
3,4,9237-HQITU,Female,No,No,No,2,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
4,5,9305-CDSKC,Female,No,No,No,8,Yes,Yes,Fiber optic,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes


In [3]:
tel = tel.drop(['customerID'], axis=1)
tel.head()

,index,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,Male,No,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
1,2,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
2,3,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
3,4,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
4,5,Female,No,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes


## Data preprocessing (5).  
This time we will skip all the exploration steps and only do some simple features preprocessing:
- Generate `tenure_group` attribute: discretize `tenure` into 6 groups: “0-12”, “12-24”, “24-36”, “36-48”, “48-60”, “60+” (all are left closed intervals: \[0, 12), \[12, 24), \[24, 36)...). What are the sizes of these groups? Tenure refers to the number of months that a customer has subscribed for. Do not drop the `tenure` column.  
- Preprocess categorical columns with only 2 unique values (“binary” columns): replace one unique value with 0 and another with 1 (label encoding). How many such columns do you have?
E.g. for the `gender` attribute you may replace Female with 1 and Male with 0 or vice versa. 
- Preprocess categorical columns with more then 2 unique values using dummy encoding (=one-hot encoding). How many such columns (before dummy encoding) do you have? 
Drop customerID attribute.

In [4]:
tel['tenure_group'] = 1
tel.loc[(12 <= tel.tenure) & (tel.tenure < 24), 'tenure_group'] = 2
tel.loc[(24 <= tel.tenure) & (tel.tenure < 36), 'tenure_group'] = 3
tel.loc[(36 <= tel.tenure) & (tel.tenure < 48), 'tenure_group'] = 4
tel.loc[(48 <= tel.tenure) & (tel.tenure < 60), 'tenure_group'] = 5
tel.loc[60 <= tel.tenure, 'tenure_group'] = 6

tel.head()

,index,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,1,Male,No,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,3
1,2,Male,No,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1
2,3,Male,No,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,4
3,4,Female,No,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1
4,5,Female,No,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1


In [5]:
tel.groupby(['tenure_group']).index.count()

tenure_group
1    1573
2     844
3     719
4     670
5     775
6    1462
Name: index, dtype: int64

In [6]:
cat_cols = tel.select_dtypes(include=['object']).columns
bin_cat_cols = [cat_col for cat_col in cat_cols if len(tel[cat_col].unique()) == 2]
nonbin_cat_cols = [cat_col for cat_col in cat_cols if len(tel[cat_col].unique()) != 2]

print(f'binary categorical columns: {len(bin_cat_cols)}')
print(f'nonbinary categorical columns: {len(nonbin_cat_cols)}')

binary categorical columns: 13
nonbinary categorical columns: 4


In [7]:
le = LabelEncoder()

for col in bin_cat_cols:
    tel[col] = le.fit_transform(tel[col])
for col in nonbin_cat_cols:
    tel = pd.get_dummies(tel, prefix=[col], columns=[col], drop_first=True)

tel.head()

,index,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,...,tenure_group,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,1,0,0,0,34,1,1,0,1,...,3,0,0,0,0,1,0,0,0,1
1,2,1,0,0,0,2,1,1,1,0,...,1,0,0,0,0,0,0,0,0,1
2,3,1,0,0,0,45,0,1,0,1,...,4,1,0,0,0,1,0,0,0,0
3,4,0,0,0,0,2,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,5,0,0,0,0,8,1,0,0,1,...,1,0,1,1,0,0,0,0,1,0


## Build a churn model (5).
- Build 2 classification models to predict customers churn:
    - Logistic Regression. What is the ROC AUC of this model?
    - Random Forest. What is the ROC AUC of this model?

*In this task I suggest you deviate from the train-test strategy and use a k-fold approach to train and predict the whole dataset: train on ⅘ of the data, predict on ⅕ , repeat this 5 times, thus you will get predictions for the whole dataset. Recall that in this case you will actually have 5 trained classifiers not 1, but for our purposes this is ok.*
After this section you must have predictions for all customers from the dataset (~6k) obtained using cross validation.

Comments:
For the section III you may use any classification model you want (you are not restricted by these two models above).
You may want to use grid search to look for the best parameters of the model/s.

## Compare two discount strategies (25 + 10).
Assumptions:
- Every customer pays the same price p which is the average of `MonthlyCharges`.
- If we decide to provide a discount we provide it to all the customers who are predicted as Churn=Yes.
- When we compute gains, costs and losses we compute them for the short term.
Therefore all the computations from the seminar hold (except you need to recompute the coefficients).
- Strategy’s profit is the difference between gains, costs and losses: 
profit = gains - costs - losses
- Profit per customer is the total profit divided by the number of customers (if the person churns the person is not a customer anymore).

**Strategy A**: Provide a 20% discount with a 75% acceptance rate.
**Strategy B**: Provide a 30% discount with a 90% acceptance rate.
In the seminar we had a 30% discount with 80% acceptance rate.

- Use the default threshold of 0.5 to compute the confusion matrix. Based on this confusion matrix report (5 points):
    - TP, FP, TN, FN
    - Losses if you do not apply any discount strategy.
    - Total gains from the discount strategy B.
    - Total costs of the discount strategy B.
    - Total losses of the discount strategy B.
    - Total profit of the discount strategy B.
    - Profit per customer pd(using strategy B). 
- Use 9 different thresholds: 0.1, 0.2, … 0.9. Answer to the following questions (10 points):
    - What is the threshold with the highest accuracy, using strategy B?
    - What is the threshold with the highest profit, using strategy B? What is the highest profit, using strategy B?
    - What is the threshold with the highest profit per customer, using strategy B ?  What is the highest profit per customer, using strategy B?
    - What is the ratio of profit per customer (obtained on the previous step) and p? pd p
    - Which strategy yields the highest profit (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit in that case?
    - Which strategy yields the highest profit per customer (A or B)? What are the TP, FP, TN, FN in this case? What is the highest profit per customer in that case?
- Prepare a report (10 points). Your report must summarize your results. Reports with simple copy paste of the results will be graded with 0 points! Some example questions (you are not limited or restricted to them): 
    - Do thresholds for the highest profit and highest profit per customer coincide or not? Why? 
    - Which you decide to choose? Under what circumstances (how many clients will you lose in both situations; what should be the decision criteria)?
    - How hard does your profit per customer decrease for customers for whom you provide a discount, compared to customers for whom you do not provide a discount? Compare this number with the discount.
- *For the bonus 10 points you need to redo all computations, but now instead of average p you should use customer’s MonthlyCharges. All the results for the bonus task must be summarized in an additional report. You must provide a comparison of the results (with the regular case when you use p).